In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, DoubleType

In [0]:
# Cargar tabla Bronze
df_bronze = spark.table("workspace.credit_risk.bronze_credit_risk")
print(f"Total de registros: {df_bronze.count():,}")

In [0]:
df_bronze = ( df_bronze
    .select(
        # Conversiones Numéricas (Importantes para análisis)
        F.col("edad").cast(IntegerType()),
        F.col("ingreso_anual").cast(DoubleType()), 
        F.col("anios_empleo").cast(DoubleType()),
        F.col("monto").cast(IntegerType()),
        F.col("tasa_interes").cast(DoubleType()),
        F.col("estado_pago").cast(IntegerType()),
        F.col("pct_ingreso").cast(DoubleType()),
        F.col("anios_hist_credito").cast(IntegerType()),
        
        # Columnas Categóricas/String (Solo se mantienen)
        F.col("tipo_vivienda"),
        F.col("proposito"),
        F.col("calificacion"),
        F.col("historial_default"),
        
        # Columna de Metadatos
        F.col("fecha_ingesta").alias("fecha_ingesta")
    )
)

In [0]:
# Mostrar primeras filas
#display(df_bronze.limit(10))

# Schema
# df_bronze.printSchema()

In [0]:
from pyspark.sql.functions import col, count, when, sum as spark_sum, round as spark_round

# Conteo de nulos y porcentaje por columna
total_rows = df_bronze.count()

null_analysis = df_bronze.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in df_bronze.columns
]).collect()[0].asDict()

# Crear DataFrame con análisis
null_df = spark.createDataFrame(
    [(k, v, round(v/total_rows*100, 2)) for k, v in null_analysis.items()],
    ["columna", "nulos", "porcentaje_nulos"]
).orderBy(col("nulos").desc())

null_df.filter(col("nulos") > 0).display()

In [0]:
# Verificar duplicados completos
total_registros = df_bronze.count()
registros_unicos = df_bronze.dropDuplicates().count()
duplicados = total_registros - registros_unicos

#print(f"Total de registros: {total_registros:,}")
#print(f"Registros únicos: {registros_unicos:,}")
print(f"Duplicados: {duplicados:,}")

if duplicados > 0:
    print(f"Hay {duplicados} registros duplicados ({round(duplicados/total_registros*100, 2)}%) del total")
else:
    print("No hay duplicados")

In [0]:
# Estadísticas para columnas numéricas
columnas_numericas = ["edad", "ingreso_anual", "anios_empleo", "monto", 
                      "tasa_interes", "pct_ingreso", "anios_hist_credito"]

display(df_bronze.select(columnas_numericas).summary())

In [0]:
# Valores únicos en columnas categóricas
print("=== TIPO DE VIVIENDA ===")
display(df_bronze.groupBy("tipo_vivienda").count().orderBy(col("count").desc()))

print("\n=== PROPÓSITO DEL PRÉSTAMO ===")
display(df_bronze.groupBy("proposito").count().orderBy(col("count").desc()))

print("\n=== CALIFICACIÓN ===")
display(df_bronze.groupBy("calificacion").count().orderBy(col("count").desc()))

print("\n=== HISTORIAL DEFAULT ===")
display(df_bronze.groupBy("historial_default").count().orderBy(col("count").desc()))

print("\n=== ESTADO DE PAGO ===")
display(df_bronze.groupBy("estado_pago").count().orderBy(col("count").desc()))

In [0]:
from pyspark.sql.functions import percentile_approx

# Detectar outliers usando percentiles
for col_name in columnas_numericas:
    stats = df_bronze.select(
        percentile_approx(col_name, 0.25).alias("Q1"),
        percentile_approx(col_name, 0.50).alias("mediana"),
        percentile_approx(col_name, 0.75).alias("Q3"),
        percentile_approx(col_name, 0.99).alias("P99")
    ).collect()[0]
    
    Q1 = stats["Q1"]
    Q3 = stats["Q3"]
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.7 * IQR
    upper_bound = Q3 + 1.7 * IQR
    
    outliers = df_bronze.filter(
        (col(col_name) < lower_bound) | (col(col_name) > upper_bound)
    ).count()
    
    print(f"{col_name}: {outliers:,} outliers ({round(outliers/total_rows*100, 2)}%)")

In [0]:
# Validaciones específicas del dominio
print("=== VALIDACIONES DE NEGOCIO ===\n")

# Edades inválidas
edad_invalida = df_bronze.filter((col("edad") < 18) | (col("edad") > 100)).count()
print(f" Edades < 18 o > 100: {edad_invalida:,}")

# Ingresos negativos o cero
ingreso_invalido = df_bronze.filter(col("ingreso_anual") <= 0).count()
print(f" Ingresos <= 0: {ingreso_invalido:,}")

# Años de empleo negativos
empleo_invalido1 = df_bronze.filter(col("anios_empleo") < 0).count()
print(f" Años empleo < 0: {empleo_invalido1:,}")

# Años de empleo negativos
empleo_invalido2 = df_bronze.filter((col("anios_empleo") < 0) | (col("anios_empleo") > 50)).count()
print(f" Años_empleo < 0 o > 50: {empleo_invalido2:,}")

# Monto préstamo <= 0
monto_invalido = df_bronze.filter(col("monto") <= 0).count()
print(f" Monto préstamo <= 0: {monto_invalido:,}")

# Tasa de interés inválida
tasa_invalida = df_bronze.filter((col("tasa_interes") < 0) | (col("tasa_interes") > 50)).count()
print(f" Tasa interés < 0 o > 50%: {tasa_invalida:,}")

# Estado de pago fuera de rango (debe ser 0 o 1)
estado_invalido = df_bronze.filter(~col("estado_pago").isin([0, 1])).count()
print(f" Estado pago diferente de 0/1: {estado_invalido:,}")

In [0]:
print("=" * 60)
print("RESUMEN DE CALIDAD DE DATOS - CAPA BRONZE")
print("=" * 60)
print(f"Total registros: {total_registros:,}")
print(f"Duplicados: {duplicados:,}")
print(f"Registros con problemas de negocio: {edad_invalida + ingreso_invalido + empleo_invalido1 + empleo_invalido2 + monto_invalido + tasa_invalida + estado_invalido:,}")
print(f"Registros válidos estimados: {total_registros - duplicados - (edad_invalida + ingreso_invalido + empleo_invalido1 + empleo_invalido2 + monto_invalido + tasa_invalida + estado_invalido):,}")